In [1]:
library(igraph)
library(dplyr)
library(Hmisc)


Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


Warning message:
“package ‘dplyr’ was built under R version 4.2.3”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:igraph’:

    as_data_frame, groups, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“package ‘Hmisc’ was built under R version 4.2.3”

Attaching package: ‘Hmisc’


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units




In [2]:
species.df = read.csv('./4.cor/rename.species.expr.csv',row.names = 1)

In [3]:
kind = c('g__Akkermansia_s__Akkermansia_muciniphila',
'g__Bacteroides_s__Bacteroides_caecimuris',
'g__Helicobacter_s__Helicobacter_hepaticus',
'g__Acutalibacter_s__Acutalibacter_muris',
'g__Lachnoclostridium_s__Lachnoclostridium_sp. YL32'
)

bad = c('g__Muribaculum_s__Muribaculum_sp. TLL-A4',
'g__Muribaculum_s__Muribaculum_sp. H5',
'g__Muribaculum_s__Muribaculum_intestinale',
'g__Duncaniella_s__Duncaniella_sp. B8',
'g__Lactobacillus_s__Lactobacillus_murinus'
)

In [4]:
species.df_t = species.df

In [5]:
trt_id <- c('KO.CLP','KO.NC','WT.CLP','WT.NC') ## 定义样本的关键词，然后从样本名抓取处理的样本
split_otu <- lapply(apply(sapply(trt_id,function(x){grep(x,colnames(species.df_t))}),2,FUN = function(x){species.df_t[,x]}),function(x){x[-(which(rowSums(x)==0)),]})

In [6]:
split_otu1 = lapply(split_otu,function(x){x[c(kind,bad),]})

In [11]:
library(readxl)

Warning message:
“package ‘readxl’ was built under R version 4.2.3”


In [38]:
data <- read.csv("../daixiezu/LM2021-12430-东部战区总医院-小鼠肠道组织/LM2021-12430-东部战区总医院-小鼠肠道组织-LCMS-EMDB(项目报告)/3.数据矩阵/metabolomics_data-new.csv",row.names = 1)

In [39]:
dim(data)

[1] 1319   32

In [40]:
head(data,2)

,KO.CLP1,KO.CLP2,KO.CLP3,KO.CLP4,KO.CLP5,KO.CLP6,KO.CLP7,KO.CLP8,KO.NC1,KO.NC2,⋯,WT.CLP7,WT.CLP8,WT.NC1,WT.NC2,WT.NC3,WT.NC4,WT.NC5,WT.NC6,WT.NC7,WT.NC8
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"LysoPC(20:4(5Z,8Z,11Z,14Z))",86674.91,218881.9,280332.4,178108.1,119883.5,32693.82,345224.2,119606,153810.9,328915.6,⋯,20661.54,104916,5804.184,133204.6,243141,277966.9,146089.6,230441.2,140556.2,298852.5
LysoPC(18:0),2009726.48,3334212.1,2450525.1,2989967.5,1876162.6,348891.23,2963804.6,2351682,2083188.4,3155221.3,⋯,263312.66,1680189,207871.058,1864903.1,2001310,3029537.8,1408620.2,2274919.8,1574928.0,2431254.0


In [41]:
trt_id <- c('KO.CLP','KO.NC','WT.CLP','WT.NC') ## 定义样本的关键词，然后从样本名抓取处理的样本

In [42]:
split_metainfo = list()
for (i in trt_id){
    split_metainfo[[i]] = dplyr::select(data,contains(i))
}

In [57]:
merge_cor = list()
otu = list()
meta = list()
p = list()
r = list()

for (i in seq(1:4)){
    otu[[i]] = split_otu1[[i]]
    meta[[i]] = split_metainfo[[i]]
    library(psych)
    merge_cor[[i]]<- corr.test(as.data.frame(t(otu[[i]])), as.data.frame(t(meta[[i]])), method = 'spearman')
    p[[i]] = merge_cor[[i]]$p
    r[[i]] = merge_cor[[i]]$r
    r[[i]][p[[i]]>0.05|abs(r[[i]])<0.2] = 0
    r[[i]][is.na(r[[i]])]=0
    p[[i]][is.na(p[[i]])]=0
    outdir = paste0(getwd(),'/6.otu-metabolite-luming')
    if(dir.exists(outdir)){
      print("dir exists")
    }else{
      dir.create(outdir)
    }
    write.csv(as.data.frame(t(r[[i]])),file = paste0(outdir,'/',trt_id[i],'.cor.r.csv'))
    write.csv(as.data.frame(t(p[[i]])),file = paste0(outdir,'/',trt_id[i],'.cor.p.csv'))
}

Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in abbreviate(colnames(r), minlength = minlength):
“abbreviate used with n

[1] "dir exists"


Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”

[1] "dir exists"


Warning message in abbreviate(colnames(r), minlength = minlength):
“abbreviate used with non-ASCII chars”
Warning message in abbreviate(dimnames(ans)[[2L]], minlength = abbr.colnames):
“abbreviate used with non-ASCII chars”
Warning message in abbreviate(dimnames(ans)[[2L]], minlength = abbr.colnames):
“abbreviate used with non-ASCII chars”
Warning message in abbreviate(dimnames(ans)[[2L]], minlength = abbr.colnames):
“abbreviate used with non-ASCII chars”


[1] "dir exists"


Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”

[1] "dir exists"
